# 일반 다층 퍼셉트론 모델(MLP)로 이미지 데이터 분류하기

이번 실습에서는 테스트용 MNIST 데이터를 96% 이상의 정확도로 분류하는 **다층 퍼셉트론 모델(MLP)**을 만들고 학습시켜보겠습니다.

이번 실습에선 10개 클래스를 가진 28x28 크기의 단순 숫자 이미지만을 모델에 학습시켰지만, 1000개 클래스를 가진 224x224 크기의 각종 사물, 동물 이미지를 이 모델에 학습시켜도 좋은 성능이 나올까요?

# 일반 다층 퍼셉트론 모델(MLP)로 이미지 데이터 분류를 위한 함수/라이브러리

- tf.keras.layers.Flatten()

2차원 데이터를 1차원 데이터로 평평하게 만들어줍니다.

- tf.keras.layers.Dense(node, activation)

 - node : 노드(뉴런) 개수
 - activation : 활성화 함수

- from tensorflow.keras.utils import to_categorical

label을 클래스화 할 수 있는 원-핫 인코딩을 해주는 모듈입니다.

- label = to_categorical(label, 10)

## 지시사항
1. MNIST 데이터 셋을 전처리하는 preprocess() 함수를 완성합니다.
2. 다층 퍼셉트론(MLP) 모델을 생성합니다. 입력층과 출력층은 다음의 지시사항을 따르고, 히든층들은 레이어 내부의 변수들을 다양하게 조정하면서 자유롭게 쌓아보세요.
    - 입력층 (Input layer)
    28x28 크기의 2차원 이미지 데이터를 1차원 데이터로 평평하게 만듭니다.
    - 출력층 (Output layer)
    10개 클래스에 대한 확률을 출력합니다. 이전 장의 실습을 떠올려보면서 어떤 활성화 함수를 써야 할 지 생각해보세요.
3. 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다. 그리고 모델의 성능을 확인해 test_acc 값이 0.96을 넘도록 해보세요.

모델에 대해 손실 함수(loss function), 최적화(optimize) 알고리즘, 평가 방법(metrics)을 자유롭게 설정해보세요. 이전 장의 실습을 참고해도 좋습니다.

## Tips!
- 모델 학습(fit) 과정에서 verbose 변수를 활용하면 모델의 학습 과정 기록을 간단하게 표현할 수 있습니다. 이후 실습에서도 이를 활용해서 학습 과정 기록을 확인하세요.

    - verbose = 0: 기록을 나타내지 않음.
    - verbose = 1: 진행 바 형태의 학습 과정 기록을 나타냄.
    - verbose = 2: epoch 당 1줄의 학습 과정 기록을 나타냄.
- MLP 모델에서는 2차원 이미지 데이터의 픽셀값을 1차원으로 쭉 피고, 그것을 Dense 레이어에 넣어서 모델을 학습시킵니다. 즉, 이 모델은 이미지의 특성을 학습하는 게 아니라 단순 Numpy 배열의 값만을 학습합니다. 그러면 이미지가 가지고 있는 특성은 의미가 없어지는 걸까요?

- 지금까지 실습에서 MNIST 데이터셋을 불러온 후 항상 255로 나누었습니다.
이를 **이미지 정규화** 라고 하며 정규화를 하는 이유는 모델이 0-255 사이의 값을 학습하는 것보다 0~1 사이로 정규화된 값을 학습할 때 학습 속도가 더 빨라지고 최적화가 잘 된다는 것이 알려져 있기 때문입니다.

- 지금까지의 실습에선 0-9 사이 값인 label을 그대로 사용해서 MNIST 숫자 분류를 진행했습니다. 즉, 실제값을 그대로 클래스로 이용해서 모델을 평가하고 테스트했습니다.
하지만 이 경우 자칫 잘못하면 0-9 사이에 ‘순위’가 매겨져 0은 0으로 label이 매겨졌기에 별로 안 중요하고, 9는 9로 label이 매겨졌기에 아주 중요하다고 모델이 판단할 수 있습니다. 하지만 대부분의 다중 클래스 분류 문제에선 각 클래스의 관계가 균등합니다.
이를 해결할 수 있는 방법으로 **‘원-핫 인코딩‘** 이 있습니다.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

#MNIST는 손으로 쓴 숫자

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

'''
지시사항 1번
   MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.
   
   전처리하는데 시간을 더 많이 쓰게 되기도 한다. 

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.
           원본은 0~255 사이의 값입니다.
           
   Step02. 0~9 사이 값인 label을 클래스화 하기 위해 
           원-핫 인코딩을 진행합니다.

'''

def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    #test_images 도 동일하게 전처리 해줘야 한다. 
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)  
    
    return train_images, test_images, train_labels, test_labels


'''
지시사항 2번
   다층 퍼셉트론(MLP) 모델을 생성합니다.

'''
def MLP():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape= (28, 28)))
    model.add(tf.keras.layers.Dense(units=128, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=256, activation='sigmoid'))
    
    # 모델 정의 마지막 레이어
    model.add(tf.keras.layers.Dense(units=10,  activation='softmax')) # node 말고 숫자만
    #unit 대신 node 써도 되고, 숫자만 써도 됨
    
    return model


'''
지시사항 3번
   모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. MLP 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
              
   Step05. 모델을 테스트하고 손실(loss)값과 
           Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 
           
           모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''

def main():
    
    # 데이터를 불러옵니다.
    train_images, test_images, train_labels, test_labels = preprocess()
    
    # 지시사항 2에서 설정한 모델을 불러옵니다.
    model = MLP()
        
    # 모델의 구조를 확인합니다.
    model.summary()
    
    
    # 컴파일러를 설정합니다.
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    # fit 함수를 사용하여 모델을 학습합니다.
    # 학습 수행 시 정보는 history에 저장합니다.
    history = model.fit(train_images, train_labels, epochs = 6, batch_size = 128, verbose=1, validation_data = (test_images, test_labels) )
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = model.evaluate(test_images, test_labels, verbose = 2)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images))
    
    Visulaize([('MLP', history)], 'loss')
    
    return test_acc


if __name__ == "__main__":
    main()

11493376/11490434 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
469/469 [==============================] - 3s 6ms/step - loss: 1.1771 - accuracy: 0.6765 - val_loss: 0.2684 - val_accuracy: 0.9221
Epoch 2/6
469/469 [=======================

C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


예측한 Test Data 클래스 :  [7 2 1 ... 4 5 6]


NameError: name 'Visulaize' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

'''
지시사항 1번
   MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.
           원본은 0~255 사이의 값입니다.
           
   Step02. 0~9 사이 값인 label을 클래스화 하기 위해 
           원-핫 인코딩을 진행합니다.

'''

def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    train_images = train_images / 255.
    test_images = test_images / 255.
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)  
    
    return train_images, test_images, train_labels, test_labels


'''
지시사항 2번
   다층 퍼셉트론(MLP) 모델을 생성합니다.

'''
def MLP():
    
    model = tf.keras.Sequential()
    ## 시작하는 layer에 input_shape을 넣을 수도 있지만 Input Layer를 추가 하는 것도 가능
    model.add(tf.keras.layers.Input(shape=(28, 28)))    # Input Layer 추가
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=16, activation='sigmoid'))
    # 마지막 layer
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    
    return model


'''
지시사항 3번
   모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. MLP 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
              
   Step05. 모델을 테스트하고 손실(loss)값과 
           Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 
           
           모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''

def main():
    
    # 데이터를 불러옵니다.
    train_images, test_images, train_labels, test_labels = preprocess()
    
    # 지시사항 2에서 설정한 모델을 불러옵니다.
    model = MLP()
    
    # 모델의 구조를 확인합니다.
    model.summary()
    
    # 컴파일러를 설정합니다.
    model.compile(loss = 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    # fit 함수를 사용하여 모델을 학습합니다.
    # 학습 수행 시 정보는 history에 저장합니다.
    history = model.fit(train_images, train_labels, epochs=10, batch_size=128, 
                        validation_data=(test_images, test_labels), verbose=2)
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images))
    
    Visulaize([('MLP', history)], 'loss')
    
    return test_acc


if __name__ == "__main__":
    main()

# 이미지 데이터 확인하기
이번 실습에서는 Numpy, PIL, tensorflow.keras 등을 이용하여 이미지를 Numpy 배열로 바꿔보고, 이를 통해 이미지가 어떻게 이루어졌는지 확인해보겠습니다.

실습에서 활용할 이미지 데이터는 data 폴더의 images 폴더 안에 10장의 이미지 중 하나의 이미지에 대해 실습을 진행하겠습니다.

이미지 데이터를 확인하기 위한 라이브러리/함수

- import PIL: 
이미지를 불러오고 처리하기 위한 라이브러리입니다.

- PIL.Image.open(path): 
이미지를 불러 옵니다.

- PIL.Image.resize(width,height): 
이미지의 크기를 조정합니다.

## 지시사항
1. PIL.Image를 이용하여 이름(경로+이름)을 바탕으로 이미지를 불러오고, 이를 리스트 images에 추가하세요.
2. 이미지의 사이즈를 조정하고 이를 Numpy 배열로 변환하세요.
3. 지금까지 완성한 함수를 이용해 main 함수를 완성하세요.

## Tips!
- 원본 이미지의 크기와 resize 후의 이미지 크기(1-3, 2-3에 해당)를 비교해보세요. 그리고 원본 이미지와 resize된 이미지(1-1, 2-1에 해당)와 비교해보세요.
- 이미지 1장에 해당하는 Numpy 배열의 크기(1-3, 2-3에 해당)와 이미지 10장에 해당하는 Numpy 배열의 크기(3에 해당)를 확인해보세요. 이미지의 수, 가로 길이, 세로 길이, RGB 차원 수는 Numpy 배열에서 차이를 확인해보세요.

In [ ]:
import  pandas as pd
import numpy as np

import PIL
import matplotlib.image as img
import matplotlib.pyplot as plt

from elice_utils import EliceUtils
elice_utils = EliceUtils()

# 이미지 목록을 불러오는 함수입니다.

def load_data(path):
    return pd.read_csv(path)

'''
지시사항 1번
   PIL.Image를 이용하여 
   이름(경로+이름)을 바탕으로 이미지를 불러오고,
   이를 리스트 'images'에 추가하는 함수를 완성합니다.
   main 함수에서 'path'와 'names' 변수를 확인해보세요.
'''

def load_images(path, names):
    
    images=[]
    for fname in names:
        file_path = path + fname
        im = PIL.Image.open(file_path)
        images.append(im)
    
    
    return images

'''
지시사항 2번
   이미지의 사이즈를 main 함수에 있는 'IMG_SIZE'로 
   조정하고, 이를 Numpy 배열로 변환하는 함수를 완성합니다.
'''

def images2numpy(images, size):
    #image는 list 형태, 이미지를 하나씩 가져와서 resize
    #w, h = size
    output = []
    
    for im in images:
        im_resize = im.resize(size)
        im_npy = np.array(im_resize) #im_resize를 넘파이로 변환
        output.append(im_npy)
    output = np.array(output)
    
    return output


# 이미지에 대한 정보를 나타내주는 함수입니다.

def sampleVisualize(np_images):

    fileName = "./data/images/1000092795.jpg"
    ndarray = img.imread(fileName)
    
    plt.imshow(ndarray)
    plt.show()    
    plt.savefig("plot.png")
    
    print("\n1-1. 'fileName' 이미지(원본): ")
    elice_utils.send_image("plot.png")
    
    print('\n1-2. Numpy array로 변환된 원본 이미지:', ndarray)
    print('\n1-3. Numpy array로 변환된 원본 이미지의 크기:', np.array(ndarray).shape)
    
    plt.imshow(np_images[0])
    plt.show()
    plt.savefig("plot_re.png")
    
    print("\n2-1. 'fileName' 이미지(resize 후): ")
    elice_utils.send_image("plot_re.png")
    
    print('\n2-2. Numpy array로 변환된 resize 후 이미지:', np_images[0])
    print('\n2-3. Numpy array로 변환된 resize 후 이미지 크기:', np.array(np_images[0]).shape)    
    
    print('\n3. Numpy array로 변환된 resize 후 이미지 10장의 크기:', np.array(np_images).shape)

'''
지시사항 3번
   main 함수를 완성하세요.
   
   Step01. 이미지를 불러오는 함수를 이용해 
           'images'를 정의합니다.
   
   Step02. 이미지를 Numpy 배열로 바꾸는 함수를 이용해
           'np_images'를 정의합니다.
'''

def main():
    
    CSV_PATH = "./data/data.csv"
    IMG_PATH = "./data/images/"
    IMG_SIZE = (300,300)
    MAX_LEN = 30
    BATCH_SIZE = 2
    
    name_caption = load_data(CSV_PATH)
    names = name_caption['file_name']
    
    print(names)
    images = load_images(path = IMG_PATH, names = names)
    np_images = images2numpy(images, size = IMG_SIZE)
    
    sampleVisualize(np_images)
    
    return images, np_images
    
if __name__=='__main__':
    main()

# Convolution Layer
실습을 통해 Convolution Layer를 구성하는 방법에 대해 알아보겠습니다.

### Tensorflow에 합성곱 신경망 (CNN)을 구성하기 위한 함수/라이브러리

- keras.layers.Conv2D(filters, kernal_size, strides, padding, activation)(image)
    - filgers : output filter의 개수
    - kernel : Convolution을 위한 kernel
    - stride : 정수 및 튜플, 리스트 형태의 값 ex) [2,2]
    - padding : SAME / VALID
    - activation : 활성화 함수
    - image : 입력 영상

## 지시사항
아래의 내용을 참고하여 **keras.layers.Conv2D()** 를 완성하세요.

매개변수	값

filter   	1

kernel_size	2

padding	  VALID

kernel_initializer	kernel_init

## Tips!
- Stride은 이미지 픽셀을 건너뛰며 합성곱 연산을 진행합니다. 이를 통해 이미지의 차원을 줄일 수 있습니다.

    - $ Output size = {{(Image - Kernel) / stride} +1}$ 

- **Padding** 은 Kernel이 이미지 경계를 벗어나지 않도록 하거나 경우에 따라 경계를 벗어나는 경우 모자라는 부분을 채워줍니다.
    - SAME : 합성곱 입력과 출력을 같게 합니다.
    - VALID : Kernel이 이미지 경계를 벗어나지 않도록 합니다.
- 관련 링크

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
https://github.com/deeplearningzerotoall/TensorFlow/blob/master/lab-11-0-cnn-basics-keras-eager.ipynb

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

def Visualize(image, x, y):
    plt.imshow(image.reshape(x,y), cmap ='Greys')
    plt.savefig('plot.png')
    elice_utils.send_image("plot.png")
    

# 임의의 3 x 3 x 1 영상을 하나 만들어줍니다.
image = np.array([[[[1],[2],[3]],
                   [[4],[5],[6]],
                   [[7],[8],[9]]]], dtype = np.float32)

# 합성곱 연산을 위해 임의의 2 x 2 x 1 커널을 하나 만들어줍니다.
kernel = np.array([[[[1.]],[[1.]]],
                      [[[1.]],[[1.]]]])


# 이미지 Shape 출력 : (num of image, width, height, channel)
print('Image shape : ', image.shape)
# 커널 Shape 출력 : (width, height, channel, num of kernel)
print('Kernel shape : ', kernel.shape)
# tf.nn.conv2d에 넣기 위해 이미지와 커널의 Shape을 위와 같이 만들었습니다.


# Gray 이미지 출력
Visualize(image, 3 ,3)

kernel_init = tf.constant_initializer(kernel)
# Convolution Layer 선언
'''
지시사항1번 
   keras.layers.Conv2D()를 완성하세요.
'''
conv2d = keras.layers.Conv2D(filters= 1, kernel_size= 2, padding= 'VALID', kernel_initializer= kernel_init)(image)
Visualize(conv2d.numpy(), 2,2)


# Max Pooling Layer
전체적인 CNN (Convolution Neural Network)를 만들기 위해서는 몇가지 추가 기능이 더 필요합니다.

대표적으로 **Max Pooling Layer** 는 일반적으로 Convolution Layer 뒤에 붙여 Conv layer의 결과(Feature Map)를 축소시키는데 사용합니다.

M x N 크기의 Pooling Filter를 적용하고, 그 안에서 가장 큰 값을 추출합니다.

이러한 작업을 통해 **Feature map의 사이즈를 줄여 연산량을 줄일 수 있으며, Feature의 개수가 줄어들기 때문에 Overfitting 방지**  할 수 있습니다.

### MaxPooling을 위한 함수/라이브러리

- keras.layers.MaxPool2D(pool_size, strides, padding)
- pool_size : Pooling filter의 크기
- strides : Filter를 적용할 간격
- padding : SAME or VALID
이번 실습에서는 Max Pooling Layer을 알아보도록 하겠습니다.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

def Visualize(image, x, y):
    plt.imshow(image.reshape(x,y), cmap ='Greys')
    plt.savefig('plot.png')
    elice_utils.send_image("plot.png")
    
# 임의의 3 x 3 x 1 영상을 하나 만들어줍니다.
image = tf.constant([[[[1],[2],[3],[4]],
                   [[4],[5],[6],[7]],
                   [[7],[8],[9],[10]],
                   [[3],[5],[7],[9]]]], dtype = np.float32)

'''
지시사항 1번
Max Pooling Layer를 선언하세요.
''' 
pool = keras.layers.MaxPool2D(pool_size=2, strides=2, padding='VALID')(image)
#padding = 'VALID'는 zero padding 이다.

print(pool.shape)
print(pool.numpy())

# 원본 영상과 Max Pooling 후 영상을 출력합니다..
Visualize(image.numpy(), 4,4)
Visualize(pool.numpy(), 2,2)


# Keras로 CNN 구현하기
이번 실습에서는 테스트용 MNIST 데이터를 95% 이상의 정확도로 분류하는 CNN 모델을 만들고 학습 시켜보겠습니다.

## Keras로 CNN을 구현하기 위한 함수/라이브러리

- tf.keras.layers.Conv2D(filters, kernel_size, activation, padding)

입력 이미지의 특징, 즉 처리할 특징 맵(map)을 추출하는 레이어입니다.

    - filters : 필터(커널) 개수
    - kernel_size : 필터(커널)의 크기
    - activation : 활성화 함수
    - padding : 이미지가 필터를 거칠 때 그 크기가 줄어드는 것을 방지하기 위해서 가장자리에 0의 값을 가지는 픽셀을 넣을 것인지 말 것인지를 결정하는 변수. SAME 또는 VALID

- tf.keras.layers.MaxPool2D(padding)

처리할 특징 맵(map)의 크기를 줄여주는 레이어입니다.

    - padding : SAME 또는 VALID

- tf.keras.layers.Flatten()

Convolution layer 또는 MaxPooling layer의 결과는 N차원의 텐서 형태입니다. 이를 1차원으로 평평하게 만들어줍니다.

- tf.keras.layers.Dense(node, activation)
    - node : 노드(뉴런) 개수
    - activation : 활성화 함수

- np.expand_dims(data, axis)

Numpy 배열 데이터에서 마지막 축(axis)에 해당하는 곳에 차원 하나를 추가할 수 있는 코드입니다.

## 지시시항
1. MNIST 데이터 셋을 전처리하는 **preprocess()** 함수를 완성합니다.
2. CNN 모델을 생성합니다. 입력층과 출력층은 다음의 지시사항을 따르고, 히든층들은 위 설명의 **CNN in Keras** 를 참고해 자유롭게 쌓아보세요. 단, Convolution 레이어 - MaxPooling 레이어 쌍이 최소 3개 이상이 되도록 하세요.

    - **입력층 (Input layer):**  
    CNN 모델에서는 입력층에서 Flatten 레이어를 이용해 데이터를 1차원으로 펴줄 필요 없이 Convolution layer를 입력층으로 쓰고, 내부 변수인 input_shape를 조정함으로써 2차원 데이터를 바로 입력으로 넣을 수 있습니다.
    이전 실습과 이번 실습 1번을 참고해서 input_shape이 어떻게 되야할 지 생각해서 입력층을 작성해보세요.

    - **출력층 (Output layer):**  
    10개 클래스에 대한 확률을 출력합니다. 이전 실습과 동일합니다.

3. 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

모델에 대해 손실 함수(loss function), 최적화(optimize) 알고리즘, 평가 방법(metrics)은 이전 실습과 동일하게 설정하세요.

모델의 성능을 확인해 test_acc 값이 0.95을 넘도록 해보세요.

## Tips!
- MNIST 데이터는 이미지 데이터이지만 가로 길이와 세로 길이만 존재하는 2차원 데이터입니다. CNN 모델은 채널(RGB 혹은 흑백)까지 고려한 3차원 데이터를 입력으로 받기에 채널 차원을 추가해 데이터의 모양(shape)을 바꿔줍니다. 결과는 아래와 같습니다.

[데이터 수, 가로 길이, 세로 길이] -> [데이터 수, 가로 길이, 세로 길이, 채널 수]
- 이전 실습의 MLP 모델과 이번 실습의 CNN 모델엔 어떤 차이가 있을지 생각해보세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *
from plotter import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

'''
지시사항 1번
MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.
   
   Step01과 Step03은 이전 실습과 동일한 코드를 사용할 수 있습니다.

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.
           원본은 0~255 사이의 값입니다.
           
   Step02. MNIST 데이터의 채널 차원을 추가해줍니다.
           
   Step03. 0~9 사이 값인 레이블을 클래스화 하기 위해 원-핫 인코딩을 진행합니다.
   
'''

def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    # Train 데이터 5000개와 Test 데이터 1000개를 사용합니다.
    train_images, train_labels = train_images[:5000], train_labels[:5000]
    test_images, test_labels = test_images[:1000], test_labels[:1000]
    
    train_images = train_images / 255
    test_images = test_images / 255
    
    print(train_images.shape, test_images.shape)
    
    train_images = np.expand_dims(train_images, axis=-1)
    test_images = np.expand_dims(test_images, axis= -1)
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_images, test_images, train_labels, test_labels

'''
지시사항 2번
CNN 모델을 생성합니다.
Convolution
'''
def CNN():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(28,28,1)))
    
    #Conv + pool 3번 이상 정도 하기, filters 수 갈수록 늘려주면 정확도 높아짐
    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='sigmoid', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(2, padding='VALID'))
    
    model.add(tf.keras.layers.Flatten())
    
    #MLP -> Dense, 학습 파라미터
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model
    
'''
지시사항 3번
모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. CNN 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
           단, 'epochs'이 클수록, 'batch_size'는 작을수록 학습 속도가 느립니다.
   
   Step05. 모델을 테스트하고 손실(loss)값과 Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''

def main():
    
    # 데이터를 불러옵니다.
    train_images, test_images, train_labels, test_labels = preprocess()
    
    # 지시사항 2에서 설정한 모델을 불러옵니다.
    model = CNN()
    
    # 모델의 구조를 확인합니다.
    model.summary()
    
    # 컴파일러를 설정합니다. optimizer는 adam으로 해도 되고...
    #optimizer = tf.keras.optimizers.Adam(0.0001) -> 이거 쓸거면 아래에 optimizer = optimizer 로 써줘야 함
    model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',metrics=['accuracy'] )
    
    # fit 함수를 사용하여 모델을 학습합니다.
    # 학습 수행 시 정보는 history에 저장합니다.
    history = model.fit(train_images, train_labels, epochs=2, batch_size=128, validation_data=(test_images, test_labels), verbose = 2 )
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images)[:10])
    
    Visulaize([('CNN', history)], 'loss')
    
    Plotter(test_images, model)
    
    return test_acc
    
if __name__ == "__main__":
    main()